In [5]:
!pip install biopython requests scholarly > /dev/null 2>&1

from Bio import Entrez
import requests
from scholarly import scholarly
import concurrent.futures

# Set email for Entrez
Entrez.email = "worthylastchance@gmail.com"

def fetch_with_timeout(func, *args, timeout=10):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(func, *args)
        try:
            return future.result(timeout=timeout)
        except concurrent.futures.TimeoutError:
            print("Request timed out.")
            return None

def search_pubmed(query, max_results=5):
    try:
        handle = fetch_with_timeout(lambda: Entrez.esearch(db="pubmed", term=query, retmax=max_results))
        if handle:
            record = Entrez.read(handle)
            print(f"PubMed Raw Result: {record}")  # Debugging output
            return record['IdList']
    except Exception as e:
        print(f"Error searching PubMed: {e}")
    return []

def fetch_article_details(article_id):
    with Entrez.efetch(db="pubmed", id=article_id, retmode="xml") as handle:
        record = Entrez.read(handle)
        article = record['PubmedArticle'][0]
        title = article['MedlineCitation']['Article']['ArticleTitle']
        abstract = article['MedlineCitation']['Article'].get('Abstract', {}).get('AbstractText', ["No abstract"])[0]
        return {'id': article_id, 'title': title, 'abstract': abstract, 'source': 'PubMed'}

def fetch_similar_articles(article_id, max_similar=5):
    similar_articles = []
    try:
        with Entrez.elink(dbfrom="pubmed", id=article_id, linkname="pubmed_pubmed", retmode="xml") as handle:
            record = Entrez.read(handle)
            links = record[0]["LinkSetDb"][0]["Link"][:max_similar]

            for link in links:
                similar_id = link["Id"]
                similar_article = fetch_article_details(similar_id)
                similar_articles.append(similar_article)

    except Exception as e:
        print(f"Error fetching similar articles: {e}")

    return similar_articles

import requests

# Fonction de recherche d'articles via CrossRef
def search_crossref(query, rows=5):
    url = f"https://api.crossref.org/works?query={query}&rows={rows}"
    response = requests.get(url)

    if response.status_code == 200:
        results = response.json()
        articles = results['message']['items']

        for article in articles:
            title = article.get('title', ['Titre non disponible'])[0]
            authors = ', '.join([author['given'] + ' ' + author['family'] for author in article.get('author', [])])
            doi = article.get('DOI', 'DOI non disponible')
            publication_date = article.get('created', {}).get('date-time', 'Date non disponible')
            url = f"https://doi.org/{doi}"

            print(f"Titre : {title}")
            print(f"Auteurs : {authors}")
            print(f"DOI : {doi}")
            print(f"Date de publication : {publication_date}")
            print(f"URL : {url}\n")
    else:
        print("Erreur lors de la récupération des articles.")


def search_google_scholar(query, max_results=5):
    results = []
    try:
        search_query = scholarly.search_pubs(query)
        for _ in range(max_results):
            article = next(search_query)
            article_bib = article.get('bib', {})
            results.append({
                'id': article_bib.get('pub_year', 'Unknown'),
                'title': article_bib.get('title', 'No title available'),
                'abstract': article_bib.get('abstract', 'No abstract available'),
                'source': 'Google Scholar'
            })
        return results
    except Exception as e:
        print(f"Error searching Google Scholar: {e}")
        return []  # Retourne une liste vide en cas d'erreur


def main(query):
    pubmed_results = search_pubmed(query)
    pubmed_details = []
    for article_id in pubmed_results:
        article_details = fetch_article_details(article_id)
        article_details['similar_articles'] = fetch_similar_articles(article_id)
        pubmed_details.append(article_details)

    crossref_results = search_crossref(query)  # Maintenant, cela devrait toujours retourner une liste
    google_scholar_results = search_google_scholar(query)  # Toujours une liste

    # Combinaison de tous les résultats
    all_results = pubmed_details + crossref_results + google_scholar_results

    for result in all_results:
        if isinstance(result, dict) and 'id' in result and 'title' in result:
            print(f"ID: {result['id']}, Title: {result['title']}\n")
            print(f"Abstract: {result['abstract']}\n")
            print(f"Source: {result['source']}\n")
            if 'similar_articles' in result:
                print("Similar Articles:")
                for similar in result['similar_articles']:
                    print(f"  - Similar ID: {similar['id']}, Title: {similar['title']}, Abstract: {similar['abstract']}")
            print()  # Ligne vide pour séparer les articles
        else:
            print("Unexpected format:", result)

# Execute search with an example query
query = "soft mist inhalers"  # Example query
main(query)

PubMed Raw Result: {'Count': '219', 'RetMax': '5', 'RetStart': '0', 'IdList': ['39568351', '39400958', '39222967', '38835808', '38817136'], 'TranslationSet': [{'From': 'inhalers', 'To': '"administration, inhalation"[MeSH Terms] OR ("administration"[All Fields] AND "inhalation"[All Fields]) OR "inhalation administration"[All Fields] OR "inhalant"[All Fields] OR "inhalability"[All Fields] OR "inhalable"[All Fields] OR "inhalants"[All Fields] OR "inhalated"[All Fields] OR "inhalation"[MeSH Terms] OR "inhalation"[All Fields] OR "inhal"[All Fields] OR "inhalations"[All Fields] OR "inhale"[All Fields] OR "inhaled"[All Fields] OR "inhaling"[All Fields] OR "inhalational"[All Fields] OR "inhalative"[All Fields] OR "inhalatively"[All Fields] OR "inhalent"[All Fields] OR "inhaler\'s"[All Fields] OR "inhales"[All Fields] OR "nebulizers and vaporizers"[MeSH Terms] OR ("nebulizers"[All Fields] AND "vaporizers"[All Fields]) OR "nebulizers and vaporizers"[All Fields] OR "inhalator"[All Fields] OR "inh

TypeError: can only concatenate list (not "NoneType") to list

In [3]:
!pip install biopython requests scholarly > /dev/null 2>&1

from Bio import Entrez
import requests
from scholarly import scholarly
import concurrent.futures

# Set email for Entrez
Entrez.email = "worthylastchance@gmail.com"

def fetch_with_timeout(func, *args, timeout=10):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(func, *args)
        try:
            return future.result(timeout=timeout)
        except concurrent.futures.TimeoutError:
            print("Request timed out.")
            return None

def search_pubmed(query, max_results=5):
    try:
        handle = fetch_with_timeout(lambda: Entrez.esearch(db="pubmed", term=query, retmax=max_results))
        if handle:
            record = Entrez.read(handle)
            print(f"PubMed Raw Result: {record}")  # Debugging output
            return record['IdList']
    except Exception as e:
        print(f"Error searching PubMed: {e}")
    return []

def fetch_article_details(article_id):
    with Entrez.efetch(db="pubmed", id=article_id, retmode="xml") as handle:
        record = Entrez.read(handle)
        article = record['PubmedArticle'][0]
        title = article['MedlineCitation']['Article']['ArticleTitle']
        abstract = article['MedlineCitation']['Article'].get('Abstract', {}).get('AbstractText', ["No abstract"])[0]
        return {'id': article_id, 'title': title, 'abstract': abstract, 'source': 'PubMed'}

def fetch_similar_articles(article_id, max_similar=5):
    similar_articles = []
    try:
        with Entrez.elink(dbfrom="pubmed", id=article_id, linkname="pubmed_pubmed", retmode="xml") as handle:
            record = Entrez.read(handle)
            links = record[0]["LinkSetDb"][0]["Link"][:max_similar]

            for link in links:
                similar_id = link["Id"]
                similar_article = fetch_article_details(similar_id)
                similar_articles.append(similar_article)

    except Exception as e:
        print(f"Error fetching similar articles: {e}")

    return similar_articles

import requests

# Fonction de recherche d'articles via CrossRef
def search_crossref(query, rows=5):
    url = f"https://api.crossref.org/works?query={query}&rows={rows}"
    response = requests.get(url)

    if response.status_code == 200:
        results = response.json()
        articles = results['message']['items']
        results_list = []

        for article in articles:
            title = article.get('title', ['Titre non disponible'])[0]
            authors = ', '.join([author['given'] + ' ' + author['family'] for author in article.get('author', [])])
            doi = article.get('DOI', 'DOI non disponible')
            publication_date = article.get('created', {}).get('date-time', 'Date non disponible')
            url = f"https://doi.org/{doi}"

            results_list.append({
                'title': title,
                'authors': authors,
                'doi': doi,
                'publication_date': publication_date,
                'url': url,
                'source': 'CrossRef'
            })
        return results_list
    else:
        print("Erreur lors de la récupération des articles.")
        return []  # Retourne une liste vide si erreur


def search_google_scholar(query, max_results=5):
    results = []
    try:
        search_query = scholarly.search_pubs(query)
        for _ in range(max_results):
            article = next(search_query)
            article_bib = article.get('bib', {})
            results.append({
                'id': article_bib.get('pub_year', 'Unknown'),
                'title': article_bib.get('title', 'No title available'),
                'abstract': article_bib.get('abstract', 'No abstract available'),
                'source': 'Google Scholar'
            })
        return results
    except Exception as e:
        print(f"Error searching Google Scholar: {e}")
        return []  # Retourne une liste vide en cas d'erreur


def main(query):
    pubmed_results = search_pubmed(query)
    pubmed_details = []
    for article_id in pubmed_results:
        article_details = fetch_article_details(article_id)
        article_details['similar_articles'] = fetch_similar_articles(article_id)
        pubmed_details.append(article_details)

    crossref_results = search_crossref(query)  # Maintenant, cela devrait toujours retourner une liste
    google_scholar_results = search_google_scholar(query)  # Toujours une liste

    # Combinaison de tous les résultats
    all_results = pubmed_details + crossref_results + google_scholar_results

    for result in all_results:
        if isinstance(result, dict) and 'id' in result and 'title' in result:
            print(f"ID: {result['id']}, Title: {result['title']}\n")
            print(f"Abstract: {result['abstract']}\n")
            print(f"Source: {result['source']}\n")
            if 'similar_articles' in result:
                print("Similar Articles:")
                for similar in result['similar_articles']:
                    print(f"  - Similar ID: {similar['id']}, Title: {similar['title']}, Abstract: {similar['abstract']}")
            print()  # Ligne vide pour séparer les articles
        else:
            print("Unexpected format:", result)

# Execute search with an example query
query = "soft mist inhalers"  # Example query
main(query)

PubMed Raw Result: {'Count': '219', 'RetMax': '5', 'RetStart': '0', 'IdList': ['39568351', '39400958', '39222967', '38835808', '38817136'], 'TranslationSet': [{'From': 'inhalers', 'To': '"administration, inhalation"[MeSH Terms] OR ("administration"[All Fields] AND "inhalation"[All Fields]) OR "inhalation administration"[All Fields] OR "inhalant"[All Fields] OR "inhalability"[All Fields] OR "inhalable"[All Fields] OR "inhalants"[All Fields] OR "inhalated"[All Fields] OR "inhalation"[MeSH Terms] OR "inhalation"[All Fields] OR "inhal"[All Fields] OR "inhalations"[All Fields] OR "inhale"[All Fields] OR "inhaled"[All Fields] OR "inhaling"[All Fields] OR "inhalational"[All Fields] OR "inhalative"[All Fields] OR "inhalatively"[All Fields] OR "inhalent"[All Fields] OR "inhaler\'s"[All Fields] OR "inhales"[All Fields] OR "nebulizers and vaporizers"[MeSH Terms] OR ("nebulizers"[All Fields] AND "vaporizers"[All Fields]) OR "nebulizers and vaporizers"[All Fields] OR "inhalator"[All Fields] OR "inh